# Clustering Crypto

In [111]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [112]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv',index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [113]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [114]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns='IsTrading', inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [115]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)

In [116]:
crypto_df.dtypes

CoinName            object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [117]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]

In [118]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = crypto_df[['CoinName']]
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [119]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [120]:
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype('float64')
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [121]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df)
x.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
# Standardize the data with StandardScaler().
x_scaled = StandardScaler().fit_transform(x)
x_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [123]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(x_scaled)

In [124]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=crypto_pca,columns=['pc1','pc2','pc3'],index=crypto_df.index)
pcs_df.head()

,pc1,pc2,pc3
42,-0.344211,0.949239,-0.511672
404,-0.327533,0.949326,-0.512272
1337,2.313050,1.594356,-0.743223
BTC,-0.133733,-1.290722,0.164817
ETH,-0.140370,-1.962495,0.312298


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [125]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


c:\Users\comai\Miniforge3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [126]:
# Initialize the K-Means model.
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 4, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 4,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 4, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 4, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 4, 1, 0, 1, 0, 1, 0,

In [127]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df,crypto_names_df],axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 


#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.344211,0.949239,-0.511672,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.327533,0.949326,-0.512272,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.313050,1.594356,-0.743223,EliteCoin,4
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.133733,-1.290722,0.164817,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.140370,-1.962495,0.312298,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.173775,-1.151536,0.034491,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.391358,1.210796,-0.479013,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.147779,-2.221641,0.473399,Monero,1
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.138810,-1.962595,0.312260,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.165967,-2.027078,0.471537,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [128]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='pc1',
    y='pc2',
    z='pc3',
    color='Class',
    symbol='Class',
    hover_name='CoinName',
    hover_data=['Algorithm'],
    width=1000,
    height=1000
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [129]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [130]:
# Print the total number of tradable cryptocurrencies.
clustered_df.shape

(532, 9)

In [131]:
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.344211,0.949239,-0.511672,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.327533,0.949326,-0.512272,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.313050,1.594356,-0.743223,EliteCoin,4
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.133733,-1.290722,0.164817,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.140370,-1.962495,0.312298,Ethereum,1


In [132]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

# https://stackoverflow.com/questions/35082140/preprocessing-in-scikit-learn-single-sample-depreciation-warning

scaler = MinMaxScaler()
tcs_rs_ft = scaler.fit_transform(clustered_df['TotalCoinSupply'].to_numpy().reshape(-1,1))
tcm_rs_ft = scaler.fit_transform(clustered_df['TotalCoinsMined'].to_numpy().reshape(-1,1))
clustered_df['TotalCoinSupply'] = tcs_rs_ft
clustered_df['TotalCoinsMined'] = tcm_rs_ft
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.344211,0.949239,-0.511672,42 Coin,0
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.327533,0.949326,-0.512272,404Coin,0
1337,X13,PoW/PoS,0.029576,3.141593e-01,2.313050,1.594356,-0.743223,EliteCoin,4
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.133733,-1.290722,0.164817,Bitcoin,1
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.140370,-1.962495,0.312298,Ethereum,1


In [134]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_data = clustered_df[['TotalCoinSupply','TotalCoinsMined','CoinName','Class']]

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

scaled_data.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,4
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [136]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

# added log scaling because it is easier to see most of the data that way
scaled_data.hvplot.scatter(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    hover_cols='CoinName',
    by='Class',
    loglog=True
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)